# Holdings

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Create instruments

In [ ]:
%%luminesce

/*

------------------
Create Instruments
------------------

Description:

    - In this query, we  load instruments from an Excel file into LUSID
    - The example assumes we have a holdings file in the luminesce-examples directory of LUSID Drive

More details:

    - https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

@@file_date = select strftime('20220301');

@instruments_from_excel =
use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=instruments
--addFileName
enduse;

-- Run instruments transformation
@instruments_for_upload =
select 
Ticker, 
Name as DisplayName, 
ISIN as Isin, 
'EQ' || Isin as ClientInternal,
SEDOL as Sedol,
Currency as DomCcy,
'Equities' as AssetClass,
'Equities' as SimpleInstrumentType
from @instruments_from_excel;

-- Upload the transformed data into LUSID
select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @instruments_for_upload;


#### Step 2: Create and inline txn property

In [ ]:
%%luminesce

/*

---------------
Create Transaction property
---------------

Description:

    - In this query, we create a new transaction property called strategy and inline this property to Luminesce

More details:

    - https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/
@@scope = select 'luminesce-examples';

-- Step 1: Define the property definitions
@newProperties =
values
('Transaction', @@scope, 'strategy', 'string');

@propertyDefinitions =
select 
Column1 as [Domain], 
Column2 as [PropertyScope], 
Column3 as [PropertyCode], 
Column3 as [DisplayName], 
'Property' as [ConstraintStyle],
'system' as [DataTypeScope],
column4 as [DataTypeCode]
from @newProperties;

-- Step 2: Load property definitions

select *
from Lusid.Property.Definition.Writer
where ToWrite = @propertyDefinitions;

-- Step 3: Configure an entity provider pair to inline properties 
-- in order to interact with them the same way as standard entity data fields

@keysToCatalog = values
('Transaction/luminesce-examples/strategy', 'strategy', True, 'A property representing SHK');

@config = select column1 as [Key], column2 as Name, column3 as IsMain, column4 as Description from @keysToCatalog;

select * from Sys.Admin.Lusid.Provider.Configure
where Provider = 'Lusid.Portfolio.Holding.Writer'
and Configuration = @config
and WriteAction = 'Modify';



#### Step 3: Create transaction portfolio

In [ ]:
%%luminesce

/*

----------------------------
Create Transaction Portfolio
----------------------------

Description:

    - In this query, we create a Transaction Portfolio in LUSID

More details:

    - https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

@@portfolioScope = select 'luminesce-examples';
@@portfolioCode1 = select 'UkEquity';

/*

Step 1: Defining base currency and creation date

*/

@@base_currency = select 'GBP';
@@created_date = select #2000-01-01#;

/*

Step 2: Define the portfolio data

*/

@create_portfolio =
select   'Transaction' as PortfolioType, 
         @@portfolioScope as PortfolioScope, 
         @@portfolioCode1 as PortfolioCode, 
         @@portfolioCode1 as DisplayName, 
         '' as Description, 
         @@created_date as Created,
         'Transaction/luminesce-examples/strategy' as SubHoldingKeys,
         @@base_currency as BaseCurrency;

/*

Step 3: Upload the portfolio into LUSID

*/

@response_create_portfolio =
select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio;

select *
from @response_create_portfolio;


#### Step 4: Adjust holdings

In [ ]:
%%luminesce

/*

---------------
Adjust holdings
---------------

Description:

    - In this query, we adjust the values of two holdings.

More details:

    - https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Defining scope and code variables
@@portfolioScope = select 'luminesce-examples';

@@portfolioCode1 = select 'UkEquity';

-- Create new table to store two holdings' data
@holding_data = 
select 
@@portfolioScope as PortfolioScope,
@@portfolioCode1 as PortfolioCode,
#2022-04-21# as EffectiveAt,
'GB0031348658' as ClientInternal,
'GBP' as CostCurrency,
100000 as Units,
'Adjust' as WriteAction
union all 
select 
@@portfolioScope as PortfolioScope,
@@portfolioCode1 as PortfolioCode,
#2022-04-21# as EffectiveAt,
'GB00BH0P3Z91' as ClientInternal,
'GBP' as CostCurrency,
500 as Units,
'Adjust' as WriteAction;

-- Write table to the portfolio
select * from 
Lusid.Portfolio.Holding.Writer
where toWrite = @holding_data;


#### Step 5: Cancel holdings

In [ ]:
%%luminesce

/*

---------------
Cancel Holdings
---------------

Description:

    - In this query, we cancel the two holdings.

More details:

    - https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Defining scope and code variables
@@portfolioScope =
select 'luminesce-examples';

@@portfolioCode1 =
select 'UkEquity';

-- Defining the attributes of a cancellation
@deletion_table = 
select 
@@portfolioScope as PortfolioScope,
@@portfolioCode1 as PortfolioCode,
#2022-04-21# as EffectiveAt,
'Cancel' as WriteAction;

-- Performs the holdings cancellation 
select * 
from Lusid.Portfolio.Holding.Writer 
where toWrite = @deletion_table;


#### Step 6: Set holdings

In [ ]:
%%luminesce

/*

-----------
Set Holding
-----------

Description:

    - In this query, we add new holdings to the portfolio
    - The example assumes we have a holdings file in LUSID Drive

More details:

    - https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Defining date of holding
@@file_date =
select strftime('20220301');

-- Defining scope and code variables
@@portfolioScope =
select 'luminesce-examples';

@@portfolioCode1 =
select 'UkEquity';

-- Loading in holdings data from Excel spreadsheet
@holdings_from_spreadsheet = 
use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=lusid_holdings
--addFileName
enduse;

-- Create new table to store holding's data
@holding_data = 
select  
@@portfolioScope as PortfolioScope,
@@portfolioCode1 as PortfolioCode,
holding_date as EffectiveAt,
InstrumentId as ClientInternal,
currency as CostCurrency,
units as Units,
strategy as strategy,
'Set' as WriteAction
from @holdings_from_spreadsheet;
        
-- Write table to the portfolio
select * from Lusid.Portfolio.Holding.Writer where toWrite = @holding_data;




#### Step 7: Call holdings

In [ ]:
%%luminesce

/*

-----------
call Holding
-----------

Description:

    - In this query, we will call the holding for the holdings that i just set

More details:

    - https://support.lusid.com/knowledgebase/category/?id=CAT-01099

*/

-- Defining scope and code variables
@@portfolioScope =
select 'luminesce-examples';

@@portfolioCode =
select 'UkEquity';

select * from Lusid.Portfolio.Holding
where PortfolioScope= @@portfolioScope
and PortfolioCode= @@portfolioCode
